In [ ]:
using Pkg; Pkg.activate("..");

using StochasticFrontiers;

In [5]:
# model specification(Half Normal)
res = sfmodel_fit(
    spec = sfspec(
        usedata("data/WangHo2010_data.csv"),
        model     = :PFEWH, 
        type      = :Prod, 
        dist      = Half(σᵤ² = :_cons), 
        σᵥ²       = :_cons, 
        hscale    = (:z1, :z2),
        ivar      = :cnum, 
        depvar    = :y, 
        frontiers = (:x1, :x2, :x3)
    ),
    options = sfopt(
        warmstart_maxIT  = 100,
        warmstart_solver = :NewtonTrustRegion, 
        main_solver      = :NewtonTrustRegion
    ), 
);


*********************************
      Model Specification      
*********************************

    true fixed effect of Wang and Ho (2010 JE)

    Yᵢₜ = αᵢ + Xᵢₜ*β + ϵᵢₜ
        where ϵᵢₜ = vᵢₜ - uᵢₜ

        further,     
            vᵢₜ ∼ N(0, σᵥ²),
            σᵥ²  = exp(log_σᵥ²)

            uᵢₜ ∼ hscaleᵢₜ * uᵢ
            hscaleᵢₜ = exp(log_hscaleᵢₜ)

            uᵢ ∼ N⁺(0, σᵤ²),
            σᵤ² = exp(log_σᵤ²)

    In the case of type(cost), "- uᵢₜ" above should be changed to "+ uᵢₜ"

*********************************
        Estimation Results 
*********************************

 Numberf Of Observations:    1220
 Log-likelihood Value:       -1363.73164
 Time Consuming:             3.0994415283203125e-6

 Converged:                  true
 Number Of Total Iterations: 22
 Iteration Limit Reached:    false

┌────────────────┬───────┬─────────┬───────────┬─────────┬──────────┬───────────┬───────────┐
│                │  Var. │   Coef. │ Std. Err. │       z │ Pr(>|z|) │ Lower 95

In [6]:
marginal, marginal_mean = sfmarginal(res)
marginal_mean

(marg_z1 = 0.015230479368983967, marg_z2 = -0.5559851768776557)

In [2]:
# model specification(Truncated Normal)


res = sfmodel_fit(
    spec = sfspec(
        usedata("data/WH2010T.csv"),
        model     = :PFEWH, 
        type      = :Prod, 
        dist      = Trun(μ = :_cons, σᵤ² = :_cons), 
        σᵥ²       = :_cons, 
        hscale    = :zit,
        ivar      = :id, 
        depvar    = :yit, 
        frontiers = (:xit,)
    ),
    options = sfopt(
        warmstart_maxIT  = 200,
        warmstart_solver = :NelderMead, 
        main_solver      = :Newton
    ), 
);


 * Model specification

    true fixed effect of Wang and Ho (2010 JE)

    Yᵢₜ = αᵢ + Xᵢₜ*β + ϵᵢₜ
        where ϵᵢₜ = vᵢₜ - uᵢₜ

        further,     
            vᵢₜ ∼ N(0, σᵥ²),
            σᵥ²  = exp(log_σᵥ²)

            uᵢₜ ∼ hscaleᵢₜ * uᵢ
            hscaleᵢₜ = exp(log_hscaleᵢₜ)

            uᵢ ∼ N⁺(0, σᵤ²),
            σᵤ² = exp(log_σᵤ²)

    In the case of type(cost), "- uᵢₜ" above should be changed to "+ uᵢₜ"


 * optimization(with warmstart) 

*********************************
       Estimation Results:
*********************************

 Numberf Of Observations:    1000
 Log-likelihood Value:       -364.19278
 Time Consuming:             0.0033910274505615234

 Converged:                  true
 Number Of Total Iterations: 203
 Iteration Limit Reached:    false

┌────────────────┬───────┬─────────┬───────────┬──────────┬──────────┬───────────┬───────────┐
│                │  Var. │   Coef. │ Std. Err. │        z │ Pr(>|z|) │ Lower 95% │ Upper 95% │
├────────────────┼───────

In [3]:
[sf_inefficiency(res) sf_efficiency(res)]

1000×2 Matrix{Float64}:
 3.61393   0.0281123
 4.53487   0.0114687
 0.688788  0.502958
 2.91923   0.0554889
 1.42213   0.242788
 1.87919   0.154474
 1.1253    0.325893
 0.611531  0.543179
 1.12981   0.324436
 2.67656   0.070418
 0.59408   0.572453
 0.411208  0.674546
 1.03147   0.396872
 ⋮         
 0.675189  0.511115
 0.997896  0.371913
 2.53337   0.0815896
 1.50748   0.223621
 1.65469   0.193392
 1.47208   0.231574
 1.78248   0.170511
 0.759429  0.469084
 1.69135   0.186528
 4.94884   0.00787078
 2.12648   0.121574
 2.71312   0.0684407

In [4]:
marginal, marginal_mean = sfmarginal(res)
marginal_mean

(marg_zit = 0.5059070624015335,)

In [8]:
std_ci = sfmarginal(
    res,
    bootstrap = true,
    R         = 10,
    seed      = 123,
);

 * bootstrap marginanl effect



Resampling: 100%|██████████████████████████████| Time: 0:00:00



Bias-Corrected 95.0% Confidence Interval:

┌─────┬──────────────────────┬─────────────────┬────────────────────┐
│     │ mean of the marginal │ std.err. of the │     bias-corrected │
│     │       effect on E(u) │     mean effect │  95.0%  conf. int. │
├─────┼──────────────────────┼─────────────────┼────────────────────┤
│ zit │              0.50591 │         0.03323 │ [0.46201, 0.56282] │
└─────┴──────────────────────┴─────────────────┴────────────────────┘

